Exercise 1
Extract images from the video


In [ ]:
!pip install opencv-python
!pip install numpy
!pip install matplotlib

In [ ]:
import cv2
import os

# Specify video path and output directory
video_path = 'your_video.mp4'  # Replace with your video path
output_dir = 'extracted_images'
os.makedirs(output_dir, exist_ok=True)

# Load the video
cap = cv2.VideoCapture(video_path)
frame_count = 0

# Loop through frames in the video
while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break
    
    # Save each frame as an image
    output_path = os.path.join(output_dir, f'frame_{frame_count:04d}.jpg')
    cv2.imwrite(output_path, frame)
    frame_count += 1

cap.release()
print(f"Extracted {frame_count} frames and saved to '{output_dir}'.")


Exercise  2
1)	Classify the images using Horse and Human dataset, and do the following:
a.	Load the dataset
b.	View the no. of testing and training images
c.	plot some images
d.	Normalizing the training data
e.	Build a convolutional neural network (ResNet) to train and testing the images
f.	Show the training and testing accuracy


In [ ]:
!pip install tensorflow

In [ ]:
import os
import matplotlib.pyplot as plt
import numpy as np
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping


In [ ]:

dataset_path = 'dataset'
train_datagen = ImageDataGenerator(rescale=1./255, validation_split=0.2)


train_generator = train_datagen.flow_from_directory(
    dataset_path,
    target_size=(224, 224),
    batch_size=32,
    class_mode='binary',
    subset='training'
)

validation_generator = train_datagen.flow_from_directory(
    dataset_path,
    target_size=(224, 224),
    batch_size=32,
    class_mode='binary',
    subset='validation'
)


print("Number of training images:", train_generator.samples)
print("Number of validation images:", validation_generator.samples)


In [ ]:

class_names = list(train_generator.class_indices.keys())
fig, ax = plt.subplots(3, 3, figsize=(10, 10))
for i in range(9):
    img, label = train_generator.next()
    ax[i//3, i%3].imshow(img[0])
    ax[i//3, i%3].set_title(class_names[int(label[0])])
    ax[i//3, i%3].axis('off')
plt.tight_layout()
plt.show()


In [ ]:

base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(224, 224, 3))


x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation='relu')(x)
predictions = Dense(1, activation='sigmoid')(x)

model = Model(inputs=base_model.input, outputs=predictions)


for layer in base_model.layers:
    layer.trainable = False


model.compile(optimizer=Adam(learning_rate=0.001), loss='binary_crossentropy', metrics=['accuracy'])


early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
history = model.fit(
    train_generator,
    epochs=20,
    validation_data=validation_generator,
    callbacks=[early_stopping]
)


In [ ]:

plt.plot(history.history['accuracy'], label='Training Accuracy')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()
plt.title('Training and Validation Accuracy')
plt.show()


In [ ]:

val_loss, val_accuracy = model.evaluate(validation_generator)
print(f'Validation Accuracy: {val_accuracy:.2f}')
